In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sparql_dataframe

# SPARQL query from Ontop
Make sure the ontop endpoint is running on `http://localhost:8080` by 
```bash
docker-compose up
```

We define the a SPARQL query to fetch the ship `mmsi`, `name` from metadata and  `lat`, `lon` from location tables.

In [ ]:
endpoint = "http://localhost:8080/sparql"

q = '''
PREFIX : <http://example.org/voc#>
PREFIX ais: <http://example.org/ais#>

SELECT DISTINCT ?time ?mmsi ?name ?lat ?lon 
WHERE {
  ?vessel a :Vessel ;
          ais:mmsi ?mmsi ;
          ais:hasMetadata ?metadata ;
          ais:hasLocation ?location .
          
  ?metadata a ais:Metadata ;
            ais:name ?name .

  ?location a ais:Location ;
            ais:lat ?lat ;
	        ais:lon ?lon ;
	        ais:time ?time .
}
'''
df = sparql_dataframe.get(endpoint, q)

Execute the query and get results as pandas dataframe.

> ⚠️ **Note:** It may take a long time to retrieve the results! I don't understand why this is since it retrieves the results immeadiatele in Protege. 

In [27]:
print(f" Number of unique MMSI: {df['mmsi'].nunique()}")
print(f"Number of rows: {len(df)}")
df.head()

 Number of unique MMSI: 673
Number of rows: 86977


,time,mmsi,name,lat,lon
0,2025-03-20T10:39:35+00:00,248020000,SOLA TS,57.316102,19.801485
1,2025-03-20T10:39:45+00:00,248020000,SOLA TS,57.316588,19.801865
2,2025-03-20T10:45:25+00:00,248020000,SOLA TS,57.334930,19.816527
3,2025-03-20T10:47:34+00:00,248020000,SOLA TS,57.341963,19.821962
4,2025-03-20T10:49:14+00:00,248020000,SOLA TS,57.347438,19.826182


Get latest data

In [28]:
latest_position = df.loc[df.groupby('mmsi')['time'].idxmax()]

print(f" Number of unique MMSI: {latest_position['mmsi'].nunique()}")
print(f"Number of rows: {len(latest_position)}")
latest_position.head()

 Number of unique MMSI: 673
Number of rows: 673


,time,mmsi,name,lat,lon
2459,2025-03-20T11:00:23+00:00,111265581,LIFEGUARD 001,59.028358,18.222660
32248,2025-03-20T11:12:37+00:00,205553000,EXEMPLAR,60.014593,23.918980
15894,2025-03-20T11:13:46+00:00,209276000,ELBWAVE,59.664528,23.556338
9064,2025-03-20T11:13:45+00:00,209313000,RIX LAKE,59.457978,24.704607
9422,2025-03-20T11:12:30+00:00,209332000,RIX PACIFIC,59.458868,24.706328
